In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
from descriptors import sift_descriptor, brisk_descriptor, keypoint_detector
from hilbert_curve import image_descriptor as hilbert_descriptor
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import time
import random

In [2]:
keys = ['sift', 'brisk', 'hilbert']
classes = ['airplanes', 'motorbikes']
#n_images = 30
percent_kp = 1

In [3]:
source_dir = Path('../caltech-101')
masks_path = source_dir / 'masks'
images_path = source_dir / 'segmented_images'

class_number = {}
i = 0

# Attribute number to class names
for classname in images_path.iterdir():
    if classname.name in classes:
        class_number[classname.name] = i
        i += 1

In [4]:
class_number

{'airplanes': 0, 'motorbikes': 1}

In [5]:
# Initialize empty lists

time_spent = {}
label = {}
data = {}

for k in keys:
    time_spent[k] = []
    label[k] = []
    data[k] = []

### Descritores e labels

In [6]:
n_kps = {}

In [7]:
for classpath in images_path.iterdir():
    
    sift = cv.SIFT_create()
    
    if classpath.name in classes:
        
        n_kps[classpath.name] = []
    
        for f in list(classpath.iterdir()):

            target = class_number[classpath.name]

            # Read image
            img_array = cv.imread(str(f), cv.IMREAD_GRAYSCALE)
            
            # Read image mask
            mask_path = masks_path / classpath.name / f.name
            mask_array = cv.imread(str(mask_path), cv.IMREAD_GRAYSCALE)

            # Detect keypoints
            keypoints = keypoint_detector(img_array, mask_array)
            sorted_kp = sorted(keypoints, key = lambda x: x.size, reverse=True)    
            
            n = int(percent_kp * len(keypoints))
            top_kp = sorted_kp[:n]
            
            n_kps[classpath.name].append(len(keypoints))


            # Compute descriptors and time spent
            t1_sift = time.time()
            sift_desc = sift_descriptor(img_array, top_kp)
            t2_sift = time.time()

            t1_brisk = time.time()
            brisk_desc = brisk_descriptor(img_array, top_kp)
            t2_brisk = time.time()

            t1_hilbert = time.time()
            hilbert_desc = hilbert_descriptor(img_array, top_kp)
            t2_hilbert = time.time()

            # Save descriptor data
            data['sift'].extend(sift_desc)
            data['brisk'].extend(brisk_desc)
            data['hilbert'].extend(hilbert_desc)

            # Save image label
            label['sift'].extend(np.asarray([target]*len(sift_desc)))
            label['brisk'].extend(np.asarray([target]*len(brisk_desc)))
            label['hilbert'].extend(np.asarray([target]*len(hilbert_desc)))

            # Save time spent
            time_spent['sift'].append(t2_sift - t1_sift)
            time_spent['brisk'].append(t2_brisk - t1_brisk)
            time_spent['hilbert'].append(t2_hilbert - t1_hilbert)

In [8]:
data['hilbert'][0]

array([52.52960881, 18.10208512, -1.0612117 , -0.96991925,  3.11634349,
        4.14497452])

### Tempo e espaço em memória 

In [9]:
np.average(np.asarray(n_kps['airplanes']))

121.88471177944862

In [10]:
np.average(np.asarray(n_kps['motorbikes']))

190.50250626566415

In [11]:
for k in keys:
    label[k] = np.asarray(label[k])
    time_spent[k] = np.asarray(time_spent[k])

In [12]:
for k in keys:
    print(k)
    print(f'time: {np.average(time_spent[k])}')
    print(f'size: {np.average(data[k][0].shape)}')
    print()

sift
time: 0.00511573012311357
size: 128.0

brisk
time: 0.02371936155142342
size: 64.0

hilbert
time: 0.3463736342308216
size: 6.0



### Classifiers

In [13]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from joblib import dump, load

In [14]:
def balance_set(data, label):
    
    rus = RandomUnderSampler(random_state=0)
    X_resampled, y_resampled = rus.fit_resample(data, label)
    
    return X_resampled, y_resampled

In [15]:
def create_model_report(classifier, X, y, print_report=False):
    
    data, label = balance_set(X, y)
    
    X_train, X_test, y_train, y_test = train_test_split(data, label)
    
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    report = classification_report(y_test, predicted, output_dict=True, target_names=['motorbike', 'airplane'])
    
    if print_report:
        print(classification_report(y_test, predicted, target_names=['motorbike', 'airplane']))
    
    return report
    

### K-Nearest Neighbours

In [16]:
keys

['sift', 'brisk', 'hilbert']

In [17]:
knn_report = {}
for key in keys:
    print(key)
    knn = KNeighborsClassifier(n_neighbors=3)
    knn_report[key] = create_model_report(knn, data[key], label[key], True)

sift
              precision    recall  f1-score   support

   motorbike       0.70      0.77      0.74     24333
    airplane       0.75      0.67      0.71     24299

    accuracy                           0.72     48632
   macro avg       0.72      0.72      0.72     48632
weighted avg       0.72      0.72      0.72     48632

brisk
              precision    recall  f1-score   support

   motorbike       0.63      0.76      0.69     20693
    airplane       0.70      0.57      0.63     21160

    accuracy                           0.66     41853
   macro avg       0.67      0.66      0.66     41853
weighted avg       0.67      0.66      0.66     41853

hilbert
              precision    recall  f1-score   support

   motorbike       0.71      0.73      0.72     15759
    airplane       0.72      0.70      0.71     15810

    accuracy                           0.72     31569
   macro avg       0.72      0.72      0.72     31569
weighted avg       0.72      0.72      0.72     31569



In [18]:
#dump(knn_sift, '../models/knn_sift.z', compress=3)
#dump(knn_brisk, '../models/knn_brisk.z', compress=3)
#dump(knn_hilbert, '../models/knn_hilbert.z', compress=3)

### Gerar tabela LaTeX com métricas dos modelos 

In [19]:
import pandas as pd

In [20]:
knn_report_all = {'sift':{}, 'brisk':{}, 'hilbert':{}}

for key in keys:
    knn_report_all[key]['keypoints'] = percent_kp
    knn_report_all[key]['accuracy'] = np.round(knn_report[key]['accuracy'], 2)

    for metric in ['precision', 'recall', 'f1-score']:
        knn_report_all[key][metric] = np.round(knn_report[key]['weighted avg'][metric], 2)
    
    knn_report_all[key]['tempo'] = np.round(np.average(time_spent[key]), 4)
    


In [21]:
df_report_all = pd.DataFrame(knn_report_all).transpose()

In [22]:
df_report_all

,keypoints,accuracy,precision,recall,f1-score,tempo
sift,1.0,0.72,0.72,0.72,0.72,0.0051
brisk,1.0,0.66,0.67,0.66,0.66,0.0237
hilbert,1.0,0.72,0.72,0.72,0.72,0.3464


In [23]:
latex_report = df_report_all.to_latex(caption = "Comparação das métricas dos modelos KNN utilizando descritores SIFT e Hilbert")

In [24]:
print(latex_report)

\begin{table}
\centering
\caption{Comparação das métricas dos modelos KNN utilizando descritores SIFT e Hilbert}
\begin{tabular}{lrrrrrr}
\toprule
{} &  keypoints &  accuracy &  precision &  recall &  f1-score &   tempo \\
\midrule
sift    &        1.0 &      0.72 &       0.72 &    0.72 &      0.72 &  0.0051 \\
brisk   &        1.0 &      0.66 &       0.67 &    0.66 &      0.66 &  0.0237 \\
hilbert &        1.0 &      0.72 &       0.72 &    0.72 &      0.72 &  0.3464 \\
\bottomrule
\end{tabular}
\end{table}

